In [67]:
# Import dependencies 
import pandas as pd
import numpy as np
import random
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt


In [70]:
# Load the news_articles.csv into a DataFrame.
stock_data_df = pd.read_csv('Resources/stock_yfinance_data.csv')
tweet_data_df = pd.read_csv('Resources/stock_tweets.csv')


In [71]:
# Display the stock data 
stock_data_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Stock Name
0,2021-09-30,260.333344,263.043335,258.333344,258.493347,258.493347,53868000,TSLA
1,2021-10-01,259.466675,260.260010,254.529999,258.406677,258.406677,51094200,TSLA
2,2021-10-04,265.500000,268.989990,258.706665,260.510010,260.510010,91449900,TSLA
3,2021-10-05,261.600006,265.769989,258.066681,260.196655,260.196655,55297800,TSLA
4,2021-10-06,258.733337,262.220001,257.739990,260.916656,260.916656,43898400,TSLA


In [72]:
stock_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6300 entries, 0 to 6299
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Date        6300 non-null   object 
 1   Open        6300 non-null   float64
 2   High        6300 non-null   float64
 3   Low         6300 non-null   float64
 4   Close       6300 non-null   float64
 5   Adj Close   6300 non-null   float64
 6   Volume      6300 non-null   int64  
 7   Stock Name  6300 non-null   object 
dtypes: float64(5), int64(1), object(2)
memory usage: 393.9+ KB


In [85]:
# Convert 'Date' column to datetime
stock_data_df['Date'] = pd.to_datetime(stock_data_df['Date'])

stock_data_df.head()

KeyError: 'Date'

In [83]:
test_stock_data_df = stock_data_df.drop(columns=['Open', 'High', 'Low', 'Adj Close', 'Volume'])


In [84]:
# Set 'Date' as the index for easy plotting
test_stock_data_df.set_index('Date', inplace=True)

# Plot the closing prices for each stock
plt.figure(figsize=(12, 8))

# Iterate over the stock columns to plot each one
for column in test_stock_data_df.columns:
    plt.plot(test_stock_data_df.index, test_stock_data_df['Stock Name'], label=column)

# Add title and labels
plt.title('Closing Prices of Multiple Stocks (2023)', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price (USD)', fontsize=12)

# Add a grid, legend, and show the plot
plt.grid(True)
plt.legend(loc='upper left')

# Show the plot
plt.show()

KeyError: "None of ['Date'] are in the columns"

In [40]:
# Display the tweet data
tweet_data_df.head()

,Date,Tweet,Stock Name,Company Name
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc."
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc."
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc."
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc."
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc."


In [41]:
tweet_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80793 entries, 0 to 80792
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          80793 non-null  object
 1   Tweet         80793 non-null  object
 2   Stock Name    80793 non-null  object
 3   Company Name  80793 non-null  object
dtypes: object(4)
memory usage: 2.5+ MB


In [42]:
tweet_data_df['Date'] = pd.to_datetime(tweet_data_df['Date'])
tweet_data_df['Date'] = tweet_data_df['Date'].dt.date

tweet_data_df.head()

,Date,Tweet,Stock Name,Company Name
0,2022-09-29,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc."
1,2022-09-29,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc."
2,2022-09-29,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc."
3,2022-09-29,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc."
4,2022-09-29,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc."


In [44]:
# Initialize the SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
sentences = tweet_data_df["Tweet"]

# Function to analyze sentiment of each sentence
def analyze_sentiment(sentences):
    return analyzer.polarity_scores(sentences)

# Apply the sentiment analysis to each tweet and create new columns
tweet_data_df[['neg', 'neu', 'pos', 'compound']] = tweet_data_df['Tweet'].apply(lambda x: pd.Series(analyze_sentiment(x)))

# tweet_data_df = tweet_data_df.drop(columns=['sentiment_scores'])

tweet_data_df.head(20)

,Date,Tweet,Stock Name,Company Name,neg,neu,pos,compound
0,2022-09-29,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc.",0.125,0.763,0.113,0.0772
1,2022-09-29,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc.",0.000,1.000,0.000,0.0000
2,2022-09-29,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc.",0.000,0.954,0.046,0.2960
3,2022-09-29,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc.",0.273,0.590,0.137,-0.7568
4,2022-09-29,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc.",0.526,0.474,0.000,-0.8750
5,2022-09-29,@RealDanODowd @Tesla This is you https://t.co/...,TSLA,"Tesla, Inc.",0.000,1.000,0.000,0.0000
6,2022-09-29,For years @WholeMarsBlog viciously silenced @T...,TSLA,"Tesla, Inc.",0.298,0.662,0.041,-0.9359
7,2022-09-29,$NIO just because I'm down money doesn't mean ...,TSLA,"Tesla, Inc.",0.155,0.690,0.155,0.2500
8,2022-09-29,50 likes for some $SPY $TSLA charts to study!\...,TSLA,"Tesla, Inc.",0.000,0.764,0.236,0.4753
9,2022-09-29,@MrJames__321 @KellyRoofing @TeslaSolar @elonm...,TSLA,"Tesla, Inc.",0.272,0.728,0.000,-0.8625


In [ ]:
# Function to normalize sentiment based on the thresholds
def normalize_sentiment(compound):
    if compound >= 0.05:
        return 1  # Positive sentiment normalized to 1
    elif -0.05 < compound < 0.05:
        return 0  # Neutral sentiment normalized to 0
    elif compound <= -0.05:
        return -1  # Negative sentiment normalized to -1

# Apply the normalization function to the 'compound_score' column
tweet_data_df['normalized_sentiment'] = tweet_data_df['compound'].apply(normalize_sentiment)

# Display the DataFrame with the normalized sentiment
tweet_data_df.head(20)

,Date,Tweet,Stock Name,Company Name,neg,neu,pos,compound,normalized_sentiment
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc.",0.125,0.763,0.113,0.0772,1
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc.",0.000,1.000,0.000,0.0000,0
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc.",0.000,0.954,0.046,0.2960,1
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc.",0.273,0.590,0.137,-0.7568,-1
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc.",0.526,0.474,0.000,-0.8750,-1
5,2022-09-29 22:25:53+00:00,@RealDanODowd @Tesla This is you https://t.co/...,TSLA,"Tesla, Inc.",0.000,1.000,0.000,0.0000,0
6,2022-09-29 22:24:22+00:00,For years @WholeMarsBlog viciously silenced @T...,TSLA,"Tesla, Inc.",0.298,0.662,0.041,-0.9359,-1
7,2022-09-29 22:23:54+00:00,$NIO just because I'm down money doesn't mean ...,TSLA,"Tesla, Inc.",0.155,0.690,0.155,0.2500,1
8,2022-09-29 22:23:28+00:00,50 likes for some $SPY $TSLA charts to study!\...,TSLA,"Tesla, Inc.",0.000,0.764,0.236,0.4753,1
9,2022-09-29 22:15:01+00:00,@MrJames__321 @KellyRoofing @TeslaSolar @elonm...,TSLA,"Tesla, Inc.",0.272,0.728,0.000,-0.8625,-1
